# 📊 Stock forecast & trading strategy with Nixtla & yFinance

This notebook shows how to analyze historical stock market data using yFinance and the Nixtla framework to derive a simple trading strategy. 

We use:
- `yFinance` to retrieve financial data
- `NeuralForecast` (RNN) for price prediction
- Fundamental data for decision support

# Setup & Imports

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../backend"))

from utils.api_data_fetcher import API_Fetcher
import asyncio
import yfinance as yf
import pandas as pd
from statsforecast import StatsForecast
from statsforecast.models import ARIMA
from datetime import datetime
from neuralforecast import NeuralForecast
from neuralforecast.models import RNN, NHITS
import ipywidgets as widgets
from IPython.display import display


Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


## Async Context: fetch all important tickers

In [ ]:
fetcher = API_Fetcher()
tickers = await fetcher.fetch_all_major_indices()


## Dropdown-Widget

Please select one of the tickers in the drop down list.

In [ ]:
ticker_dropdown = widgets.Dropdown(
    options=tickers,
    description='Ticker:',
    value='AAPL',
    style={'description_width': 'initial'}
)

display(ticker_dropdown)


Dropdown(description='Ticker:', options=('AAPL', 'AMGN', 'AMZN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DI…

## Get historical data for selected ticker

In [8]:
selected_ticker = ticker_dropdown.value
data = await fetcher.fetch_selected_stock_data_yf(selected_ticker)


## Historical price trends


In [9]:
data['price_history'].head()

Date
2015-06-01 00:00:00-04:00    28.128998
2015-07-01 00:00:00-04:00    27.202801
2015-08-01 00:00:00-04:00    25.287622
2015-09-01 00:00:00-04:00    24.847908
2015-10-01 00:00:00-04:00    26.920437
Name: Close, dtype: float64


# Data preparation

In [ ]:
df = data['price_history'].reset_index() 
df.columns = ['ds', 'y']  # Rename columns to match Nixtla's required format, 'ds' = datetime and 'y' = value
df['unique_id'] = selected_ticker  # Important for multi-series models in Nixtla
 
# Add features to the DataFrame
df['recommendation'] = 1 if data['recommendation_key'] == 'buy' else 0 # Binary Feature
df['eps_forward'] = data['eps_forward']
df['revenue_growth'] = data['revenue_growth']
df['recommendation_mean'] = data['recommendation_mean']
df['gross_margins'] = data['gross_margins']
df['dividend_yield'] = data['dividend_yield']
df['debt_to_equity'] = data['debt_to_equity']

df.tail()


# Model training & forecast

In [ ]:
# Create a recurrent neural network for forecasting
model = RNN(h=5, input_size=12, max_steps=100)

nf = NeuralForecast(
    models=[model],
    freq='M'  # Monthly data
)

# Fit model on prepared dataset
nf.fit(df=df)

# Make prediction
forecast = nf.predict()
forecast.head()

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | hist_encoder | RNN           | 49.8 K | train
4 | mlp_decoder  | MLP           | 16.6 K | train
-------------------------------------------------------
66.4 K    Trainable params
0         Non-trainable params
66.4 K    Total params
0.266     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\utilsforecast\processing.py:384: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\utilsforecast\processing.py:440: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

,unique_id,ds,RNN
0,AAPL,2025-05-31 00:00:00-04:00,256.806641
1,AAPL,2025-06-30 00:00:00-04:00,250.945084
2,AAPL,2025-07-31 00:00:00-04:00,236.336563
3,AAPL,2025-08-31 00:00:00-04:00,221.365433
4,AAPL,2025-09-30 00:00:00-04:00,229.515091


# Forecast analysis

In [ ]:
# Last known real price
current_price = df['y'].iloc[-1]

# Average of the forecasted prices
mean_forecast_price = forecast['RNN'].mean()

# Calculate absolute and percentage difference
price_diff = mean_forecast_price - current_price
percent_change = (price_diff / current_price) * 100

print(f"Current: {current_price:.2f}, Forecast: {mean_forecast_price:.2f}, Change: {percent_change:.2f}%")

Current: 205.60, Forecast: 238.99, Change: 16.24%


# Trading strategy

In [16]:
def trading_strategy(percent_change):
    if percent_change > 5:
        return f"📈 BUY CALL: Expected increase of {percent_change:.2f}% - use possible opportunity!"
    elif percent_change < -5:
        return f"📉 BUY PUT: Expected decrease of {abs(percent_change):. 2f}% - hedge risk!"
    elif -2 <= percent_change <= 2:
        return f"➖ HOLD / SELL COVERED CALL: Movement below 2% - sideways market."
    else:
        return f"🔁 STRADDLE / SPREAD: Movement possible, direction unclear (±{percent_change:.2f}%)"

print(trading_strategy(percent_change))

📈 BUY CALL: Expected increase of 16.24% - use possible opportunity!
